In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [3]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from preprocessing import *

In [4]:
print("loading pre trained embeddings, this can take some minutes...")
glove_twitter_25 = KeyedVectors.load_word2vec_format('glove-twitter-25.txt', binary=False)
print("loading complete.")

loading pre trained embeddings, this can take some minutes...
loading complete.


In [4]:
w2v_model = Word2Vec.load("w2vPreTrained")

In [5]:
w2v_model.wv.most_similar("love")

[('enjoy', 0.5578348636627197),
 ('great', 0.5526167154312134),
 ('amaze', 0.5221312046051025),
 ('like', 0.4955675005912781),
 ('awesome', 0.43842583894729614),
 ('favorite', 0.42576074600219727),
 ('fun', 0.4234786629676819),
 ('amazing', 0.4115583598613739),
 ('fantastic', 0.408966600894928),
 ('nice', 0.3966287672519684)]

In [6]:
w2v_model.wv.most_similar("hate", topn = 10)

[('complain', 0.38117867708206177),
 ('bad', 0.37489137053489685),
 ('assist', 0.3195871114730835),
 ('throw', 0.2934301197528839),
 ('hide', 0.2910609841346741),
 ('rid', 0.28255414962768555),
 ('practically', 0.28100088238716125),
 ('automate', 0.2786242365837097),
 ('handsfree', 0.278286874294281),
 ('turn', 0.2732694745063782)]

In [7]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t", encoding = "utf-8")
print(dataset.shape)
dataset.dropna(inplace = True)
print(dataset.shape)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True)
print(dataset.shape)
dataset.drop_duplicates(inplace = True)
print(dataset.shape)

(3150, 5)
(3150, 5)
(2998, 5)
(2322, 5)


In [8]:
X = np.array(dataset["verified_reviews"].values).reshape(-1, 1)
y = list(dataset["feedback"].values)

In [9]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state = 0)

X, y = undersampler.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({1: 442, 0: 221})


In [10]:
X_temp = []

for rev in X:
  X_temp.append(rev[0])

In [11]:
new_text, new_sent_tok = tokenize_list_of_text(X_temp, custom_stopwords, False, pos_list)

total number of types extracted is: 1810


In [12]:
# negative_reviews = list(dataset[dataset["feedback"] == 0]["verified_reviews"].values)
# print(len(negative_reviews))
# negative_text, negative_tok = tokenize_list_of_text(negative_reviews, custom_stopwords, False, pos_list)

In [13]:
# negative_artifical = generate_samples(negative_tok, 200, pre_trained_model = glove_twitter_25)
# new_sent_tok.extend(negative_artifical)
# len(negative_artifical)

In [14]:
def frequency_cleaning(new_sent_tok, n):
    
    tot_tokens = []

    for sent in new_sent_tok:
        for tok in sent:
            tot_tokens.append(tok)

    freqs = nltk.FreqDist(tot_tokens)
    cleaned_reviews = []

    for sent in new_sent_tok:
        clean_sent = []
        for tok in sent:
            if freqs[tok] > n:
                clean_sent.append(tok)
        cleaned_reviews.append(clean_sent)

    return cleaned_reviews

In [15]:
cleaned_reviews = frequency_cleaning(new_sent_tok, 2)
# negative_artificial_cleaned = cleaned_reviews[-len(negative_artifical):] # estraggo le recensioni artificiali per poi aggiungerle esclusivamente al train
# del cleaned_reviews[-len(negative_artifical):] # le elimino dalle recensioni pulite

In [16]:
bigrams = Phrases(cleaned_reviews, scoring="npmi", threshold=0.60) #estrae le collocazioni tramite PMI

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], y, test_size=0.20, random_state=10)
# X_train.extend(negative_artificial_cleaned)
# Y_train.extend([0 for x in range(len(negative_artificial_cleaned))])

In [18]:
print('Train set is %s' % Counter(Y_train))
print('Test set is %s' % Counter(Y_test))

Train set is Counter({1: 351, 0: 179})
Test set is Counter({1: 91, 0: 42})


In [19]:
t = Tokenizer()
t.fit_on_texts(X_train)
X_train_encoded = t.texts_to_sequences(X_train)
max_length = len(max(bigrams[cleaned_reviews], key = len))
Xtrain = pad_sequences(X_train_encoded, maxlen=max_length, padding='post')

In [20]:
X_test_encoded = t.texts_to_sequences(X_test)
Xtest = pad_sequences(X_test_encoded, maxlen=max_length, padding='post')

In [21]:
vocab_size = len(t.word_index) + 1

In [22]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, w2v_model.vector_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    not_found = 0
    not_found_list = []
    for word, i in vocab.items():
        try:
            vector = embedding[word]
            weight_matrix[i] = vector
        except KeyError:
            # weight_matrix[i] = np.zeros((1, w2v_model.vector_size))
            not_found+=1
            not_found_list.append(word)
            continue

    print(not_found_list)
    print(not_found)
    return weight_matrix

In [23]:
embedding_weights = get_weight_matrix(w2v_model.wv, t.word_index)

['play_music', 'easy_set', 'sound_quality', 'alarm_clock', 'ask_question', "'ve", 'still_learn', 'send_back', 'bluetooth_speaker', 'brand_new', 'touch_screen', 'answer_question', 'excitement', 'surrender', 'even_though', "n't_see", "'ll", "'re", "n't_tell", 'literate', 'homescreen', "n't_use", 'certified', 'impotent']
24


In [24]:
from imblearn.under_sampling import RandomUnderSampler, TomekLinks


undersampler_nn = TomekLinks(sampling_strategy="all") # use tomeLinks to remove ambiguous data through nearest neighbours


# Fit and transform the X and y data
X_train_resampled, y_train_resampled = undersampler_nn.fit_resample(Xtrain, Y_train)
# X_train_resampled, y_train_resampled = Xtrain, Y_train


print('Resampled dataset shape %s' % Counter(y_train_resampled))

Resampled dataset shape Counter({1: 325, 0: 153})


In [25]:
from keras.layers import Flatten

In [26]:
from keras.utils import to_categorical
Y_train_hot = to_categorical(y_train_resampled)

In [27]:
print(y_train_resampled[0], Y_train_hot[0])
print(y_train_resampled[1], Y_train_hot[1])

0 [1. 0.]
1 [0. 1.]


In [28]:
Y_test_hot = to_categorical(Y_test)

In [49]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Embedding, Conv1D, MaxPooling1D

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=25)

model = Sequential()
model.add(Embedding(vocab_size, w2v_model.vector_size, input_length=max_length, trainable = True, weights = [embedding_weights]))
model.add(Dropout(0.2))
model.add(Conv1D(filters=16, kernel_size=10, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 98, 100)           61900     
                                                                 
 dropout_10 (Dropout)        (None, 98, 100)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 89, 16)            16016     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 44, 16)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 704)               0         
                                                                 
 dropout_11 (Dropout)        (None, 704)               0         
                                                      

In [50]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# fit network
model.fit(X_train_resampled, Y_train_hot, epochs=100, verbose=2, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
14/14 - 2s - loss: 0.6581 - binary_accuracy: 0.6628 - val_loss: 0.6759 - val_binary_accuracy: 0.6250 - 2s/epoch - 139ms/step
Epoch 2/100
14/14 - 0s - loss: 0.5189 - binary_accuracy: 0.7756 - val_loss: 0.6968 - val_binary_accuracy: 0.6354 - 341ms/epoch - 24ms/step
Epoch 3/100
14/14 - 0s - loss: 0.4351 - binary_accuracy: 0.7988 - val_loss: 0.6646 - val_binary_accuracy: 0.7292 - 384ms/epoch - 27ms/step
Epoch 4/100
14/14 - 0s - loss: 0.3436 - binary_accuracy: 0.8291 - val_loss: 0.4251 - val_binary_accuracy: 0.7917 - 395ms/epoch - 28ms/step
Epoch 5/100
14/14 - 0s - loss: 0.2370 - binary_accuracy: 0.8907 - val_loss: 0.5076 - val_binary_accuracy: 0.8125 - 385ms/epoch - 27ms/step
Epoch 6/100
14/14 - 0s - loss: 0.1886 - binary_accuracy: 0.9128 - val_loss: 0.6005 - val_binary_accuracy: 0.7917 - 415ms/epoch - 30ms/step
Epoch 7/100
14/14 - 0s - loss: 0.1768 - binary_accuracy: 0.9093 - val_loss: 0.5228 - val_binary_accuracy: 0.7708 - 417ms/epoch - 30ms/step
Epoch 8/100
14/14 - 0s - loss

In [51]:
predictions = model.predict(Xtest)

5/5 [==============================] - 0s 3ms/step


In [52]:
predictions.shape

(133, 2)

In [53]:
predictions = np.round(predictions)

In [54]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_hot, predictions))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81        42
           1       0.92      0.90      0.91        91

   micro avg       0.88      0.88      0.88       133
   macro avg       0.86      0.87      0.86       133
weighted avg       0.88      0.88      0.88       133
 samples avg       0.88      0.88      0.88       133



In [55]:
x = ["The product is disappointing, i bought it expecting to be user friendly and intelligent, instaed it's not easy and it's stupid, too slow"]
review, temp = tokenize_list_of_text(x)
seq_review = t.texts_to_sequences(temp)
padded_review = pad_sequences(seq_review, maxlen=max_length, padding='post')
preds = np.round(model.predict(padded_review))
preds

total number of types extracted is: 14
1/1 [==============================] - 0s 27ms/step


array([[1., 0.]], dtype=float32)